# Example Notebook for DPNeurifyFV

To make sure the notebook runs with the `DPNeurifyFV` environment, activate the directory with the corresponding `Project.toml` in Julia's package manager.

**General note**: Julia precompiles a function, when it is first used. This may lead to the first execution to be rather slow. It should be much faster, for the second execution.

In [1]:
import Pkg
Pkg.activate(".")
Pkg.status()

  Activating environment at `~/VerifyNN/DPNeurifyFV/Project.toml`


     Project DPNeurifyFV v0.1.0
      Status `~/VerifyNN/DPNeurifyFV/Project.toml`
  [864edb3b] DataStructures v0.18.11
  [b4f0291d] LazySets v1.56.3
  [7e1232c4] NeuralPriorityOptimizer v0.1.0 `https://github.com/sisl/NeuralPriorityOptimizer.jl#main`
  [146f25fa] NeuralVerification v0.1.0 `https://github.com/phK3/NeuralVerification.jl#BuildingBranch`
  [d96e819e] Parameters v0.12.3
  [37e2e46d] LinearAlgebra


Import the necessary packages `DPNeurifyFV`, `NeuralVerification` and `LazySets`.

In [48]:
using NeuralVerification, DPNeurifyFV, LazySets

We are going to demonstrate the functionality on the example of the first ACAS-Xu network operating on the input defined in property $\phi_1$.

After loading the network, we convert it into a format that stores the negative and positive weights separately to make the symbolic forward pass with `DPNeurifyFV` more efficient.

In [5]:
input_set, output_set = DPNeurifyFV.get_acas_sets(1)

acas = read_nnet("./networks/ACASXU_experimental_v2a_1_1.nnet")
acas_npi = NetworkNegPosIdx(acas);

## Single Forward Pass

We demonstrate the effect of introducing the maximum number of fresh variables (the original idea is given in the [NeuroDiff paper](https://arxiv.org/abs/2009.09943)) at the first possible neurons and how it can be improved by 
- introducing at fresh variables for at most a certain fraction of the neurons in a layer and
- selecting the neurons for which to introduce fresh variables by the range between their concrete lower and upper bounds.

The effect of the techniques is visible as the computed lower and upper bounds of the $5$ output neurons of the network get progressively tighter.

Bounds computed by zonotope propagation are also shown for comparison.

In [63]:
max_vars = 0
s = init_symbolic_interval_fvheur(acas_npi, input_set, max_vars=max_vars)
ŝ₀ = forward_network(DPNFV(method=:DeepPolyRelax), acas_npi, s);

In [64]:
max_vars = 20  # best possible value found with linear search
s = init_symbolic_interval_fvheur(acas_npi, input_set, max_vars=max_vars)
ŝ₁ = forward_network(DPNFV(method=:DeepPolyRelax), acas_npi, s);

In [65]:
max_vars = 43  # best possible value found with linear search
s = init_symbolic_interval_fvheur(acas_npi, input_set, max_vars=max_vars)
ŝ = forward_network(DPNFV(method=:DeepPolyRelax, get_fresh_var_idxs=DPNeurifyFV.fresh_var_range_non_zero, var_frac=0.2), acas_npi, s);

In [66]:
# Zonotope propagation
ẑ = forward_network(Ai2z(), acas, input_set);

In [67]:
println("---- lower bounds ----")
println("no fresh vars: ", ŝ₀.lbs[end])
println("earliest vars: ", ŝ₁.lbs[end])
println("DPNeurifyFV:   ", ŝ.lbs[end])
println("Zonotopes:     ", low(ẑ))

println("---- upper bounds ----")
println("no fresh vars: ", ŝ₀.ubs[end])
println("earliest vars: ", ŝ₁.ubs[end])
println("DPNeurifyFV:   ", ŝ.ubs[end])
println("Zonotopes:     ", high(ẑ))

---- lower bounds ----
no fresh vars: [-945.7437378443676, -1593.1292720792367, -1107.327504883211, -2657.431833411753, -1721.522667184016]
earliest vars: [-713.5475439388757, -1206.8144662819318, -836.4005121250489, -2011.2980832629394, -1298.6134640550054]
DPNeurifyFV:   [-545.8439243441478, -849.9244688863708, -620.4441380554525, -1369.794720592327, -968.0683908576995]
Zonotopes:     [-3284.187439643944, -3858.877872030732, -3974.832966356989, -4135.3674257450475, -3829.9250811223164]
---- upper bounds ----
no fresh vars: [2627.2788360915074, 3430.996100304105, 3485.8259676663447, 3830.7907683913627, 3815.8806839777662]
earliest vars: [1986.046102724299, 2595.4574974837446, 2635.617754849305, 2896.4768664566177, 2885.8584796706364]
DPNeurifyFV:   [1327.6136788440915, 1711.7279892887761, 1682.6841791238921, 1985.863498730014, 1921.661177949215]
Zonotopes:     [5023.28774060603, 5701.11965091425, 6211.210325766039, 5365.67801572021, 5514.254478245289]


# Branch and Bound with Input Splitting

We are now going to maximize the first output of the network over the input space given by property $\phi_1$. (We terminate as soon as an optimality gap of $10^{-4}$ is reached)

In [51]:
params = DPNeurifyFV.PriorityOptimizerParameters(max_steps=5000, print_frequency=100, stop_frequency=1, verbosity=2)
optimize_linear_deep_poly(acas, input_set, [1.,0,0,0,0], params, solver=DPNFV(method=:DeepPolyRelax, max_vars=15), concrete_sample=:BoundsMaximizer, split=DPNeurifyFV.split_important_interval)

i: 100 - [-0.017807535560300215, 0.28346228356969677], 0.1857128143310547 sec
i: 200 - [-0.017807535560300215, -0.0005626577176235797], 0.3333280086517334 sec
i: 300 - [-0.017807535560300215, -0.014229881669890589], 0.47218894958496094 sec
i: 400 - [-0.017807535560300215, -0.01616575874515486], 0.6078557968139648 sec
i: 500 - [-0.017768583207969105, -0.01700112607245889], 0.7410039901733398 sec
i: 600 - [-0.01771721730558143, -0.017293899238105828], 0.8720948696136475 sec
i: 700 - [-0.01768762570130527, -0.017444203723618303], 0.998323917388916 sec
i: 800 - [-0.01768762570130527, -0.017532476976106312], 1.119107961654663 sec
i: 900 - [-0.01768762570130527, -0.01757186146113745], 1.237293004989624 sec
i: 1000 - [-0.01768762570130527, -0.017591160922579428], 1.3489699363708496 sec
i: 1059 - [-0.01768762570130527, -0.017602731762479756], 1.42289400100708 sec


([0.5999999999999999, 0.0009765625, 0.34375, 0.5, -0.45], -0.01768762570130527, -0.017602731762479756, 1059)

To verify, if the output of a neural network is contained within a certain polytope, we maximize the maximum violation of its constraints over the input region of interest.

For property $\phi_1$, we want to prove that the first output $o_1$ of the network is at most $b = 3.9911...$. The violation of this constraint is then $o_1 - b$. If we find an upper bound on this violation that is always below zero, then the property is satisfied.

In [72]:
input_set, output_set = DPNeurifyFV.get_acas_sets(1)

params = DPNeurifyFV.PriorityOptimizerParameters(max_steps=5000, print_frequency=10, stop_frequency=1, verbosity=2)
x_star, lower_bound, upper_bound, steps = contained_within_polytope_deep_poly(acas, input_set, output_set, params, solver=DPNFV(method=:DeepPolyRelax, max_vars=20), 
                                                                              split=DPNeurifyFV.split_important_interval, concrete_sample=:BoundsMaximizer)

@show lower_bound
@show upper_bound
@show steps;

i: 10 - [-4.010900538960003, 161.65152125857605], 0.01834392547607422 sec
i: 20 - [-4.009413609141843, 35.005185303363035], 0.05038785934448242 sec
i: 30 - [-4.00917068264572, 12.449902203700105], 0.06676292419433594 sec
i: 40 - [-4.00917068264572, 4.5827470329762505], 0.08274698257446289 sec
Returning early because of upper bound threshold
lower_bound = -4.00917068264572
upper_bound = -0.15106125856841235
steps = 44


To verify, that the output of a neural network never reaches a certain polytope, we minimize the minimum violation of its constraints over the input region of interest.

For property $\phi_2$, we want to prove that the first output $o_1$ is never larger than the other outputs of the network.
This can be captured by the linear constraints $o_1 \leq o_i$ for $i = 2,...,5$ with their respective violation $o_1 - o_2$.

If we find a lower bound that is larger than zero on the minimum of the constraint violations, we can guarantee that the polytope is never reached.

In [75]:
input_set, output_set = DPNeurifyFV.get_acas_sets(2)

params = DPNeurifyFV.PriorityOptimizerParameters(max_steps=5000, print_frequency=50, stop_frequency=1, verbosity=2)
x_star, lower_bound, upper_bound, steps = reaches_polytope_deep_poly(acas, input_set, output_set.X, params, solver=DPNFV(method=:DeepPolyRelax, max_vars=20), 
                                                                      split=DPNeurifyFV.split_important_interval, concrete_sample=:BoundsMaximizer)

@show lower_bound
@show upper_bound
@show steps

i: 50 - [-0.002872276893186433, 1.211565045737391], 0.10503101348876953 sec
i: 100 - [-0.0025448432264916516, 0.1619738574082134], 0.19298195838928223 sec
i: 150 - [-0.0025448432264916516, 0.0366643163048731], 0.26526808738708496 sec
i: 200 - [-0.0025448432264916516, 0.008490718747280575], 0.3477649688720703 sec
i: 250 - [-0.0025448432264916516, 0.0012385253647943031], 0.4178740978240967 sec
i: 300 - [-0.0025448432264916516, 9.35084304466878e-6], 0.4970970153808594 sec
Returning early because of upper bound threshold
lower_bound = 2.7476926375185132e-5
upper_bound = 0.0025448432264916516
steps = 301


301